In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
# Importo libreria para trabajar con dates 
from datetime import timedelta
# Libreria para estimar regresión
import statsmodels.formula.api as sm
# Libreria para normalizar
from sklearn.preprocessing import StandardScaler

In [4]:
import pandas as pd

# Leer los archivos Excel
c1 = pd.read_excel('c1.xlsx', sheet_name='1')
c2 = pd.read_excel('c2.xlsx', sheet_name='2')

# Crear la columna 'Key' en ambos DataFrames
def create_key_column(df, column_name):
    df[column_name] = df[column_name].astype(str).str.strip()
    df['Key'] = df[column_name].str.lower().str.split()
    df['Key'] = df['Key'].apply(lambda x: sorted(x) if isinstance(x, list) else x)
    df['Key'] = df['Key'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
    return df

c1 = create_key_column(c1, 'Asegurado')
c2 = create_key_column(c2, 'Nombre')

# Realizar la unión de los DataFrames
merged_df = pd.merge(c1, c2, on='Key', how='left', suffixes=('_c1', '_c2'))

# Eliminar filas donde no haya coincidencia en c2
merged_df = merged_df[~merged_df['Nombre'].isna()]

# Guardar el resultado en un nuevo archivo Excel
merged_df.to_excel('resultado.xlsx', index=False)

print("La unión de los archivos se ha completado y se ha guardado en 'resultado.xlsx'.")


La unión de los archivos se ha completado y se ha guardado en 'resultado.xlsx'.


In [5]:
# Guardar el resultado en un nuevo archivo Excel
merged_df.to_excel('resultado.xlsx', index=False)

print("La unión de los archivos se ha completado y se ha guardado en 'resultado.xlsx'.")


La unión de los archivos se ha completado y se ha guardado en 'resultado.xlsx'.


In [6]:
# Lista de nombres de las columnas a eliminar porque no me sirven
columnas_a_eliminar = [
    'Cuit',
    'Productor_c1',
    'Nombre Productor',
    'Certificado',
    'Fecha Emisión Endoso',
    'Fecha Vigencia Endoso',
    'Rol',
    'Productor póliza',
    'Remesa Self',
    'Remesa productor',
    'Key',
    'Nombre',
    'Empleados',
    'Dateros',
    'Grupo'
]

# Eliminar las columnas del DataFrame
merged_df.drop(columns=columnas_a_eliminar, inplace=True)

# Verificar que las columnas se han eliminado
print("Columnas después de eliminar:")
print(merged_df.columns)


Columnas después de eliminar:
Index(['Asegurado', 'Fecha ingreso', 'Mes-ano', 'Ingreso', 'Origen Ingreso',
       'Ramo', 'Póliza', 'Endoso', 'Provincia', 'Moneda', 'Prima', 'Premio',
       'Comisión normal', 'Comisión cobranza', 'Comisión fomento',
       'Comisión otras', 'Comisión total', 'Cobrador', 'Tipo Persona', 'CUIT',
       'DNI', 'Nacimiento', 'Direcciones', 'Telefonos', 'Mails', 'Ramos',
       'Productor_c2', 'Creacion', 'id'],
      dtype='object')


In [7]:
# Guardar el DataFrame resultante en un nuevo archivo Excel
merged_df.to_excel('resultado_final.xlsx', index=False)

print("El nuevo archivo Excel 'resultado_final.xlsx' ha sido generado exitosamente.")


El nuevo archivo Excel 'resultado_final.xlsx' ha sido generado exitosamente.


In [9]:
# Recordemos las columnas relevantes y sus números:

# 19. Tipo Persona
# 20. CUIT
# 30. sexo (nueva columna que vamos a crear)

# Asegurarnos de que las columnas 'Tipo Persona' y 'CUIT' sean de tipo string y eliminar espacios adicionales
merged_df['Tipo Persona'] = merged_df['Tipo Persona'].astype(str).str.strip().str.lower()  # Columna 19
merged_df['CUIT'] = merged_df['CUIT'].astype(str).str.strip()  # Columna 20

# Inicializar la columna 'sexo' (columna 30) con un valor por defecto
merged_df['sexo'] = 'Desconocido'  # Columna 30

# Aplicar las condiciones para asignar los valores correspondientes

# Condición 1: Si 'Tipo Persona' (columna 19) es 'jurídica' o 'juridica', asignar 'empresa' en 'sexo' (columna 30)
cond_empresa = merged_df['Tipo Persona'].isin(['jurídica', 'juridica'])
merged_df.loc[cond_empresa, 'sexo'] = 'empresa'

# Condición 2: Si 'Tipo Persona' no es 'empresa' y CUIT (columna 20) comienza con '20', asignar 'Masculino' en 'sexo'
cond_masculino = (~cond_empresa) & (merged_df['CUIT'].str.startswith('20'))
merged_df.loc[cond_masculino, 'sexo'] = 'Masculino'

# Condición 3: Si 'Tipo Persona' no es 'empresa' y CUIT (columna 20) comienza con '23' o '27', asignar 'Femenino' en 'sexo'
cond_femenino = (~cond_empresa) & (merged_df['CUIT'].str.startswith(('23', '27')))
merged_df.loc[cond_femenino, 'sexo'] = 'Femenino'


In [13]:
# Guardar el DataFrame resultante en un nuevo archivo Excel
merged_df.to_excel('resultado_final.xlsx', index=False)

print("El nuevo archivo Excel 'resultado_final.xlsx' ha sido generado exitosamente.")


El nuevo archivo Excel 'resultado_final.xlsx' ha sido generado exitosamente.


In [14]:
# Obtener la cantidad en cada categoría de la columna 'sexo'
conteo_sexo = merged_df['sexo'].value_counts()

# Mostrar el resultado
print("Cantidad en cada categoría de la columna 'sexo':")
print(conteo_sexo)


Cantidad en cada categoría de la columna 'sexo':
sexo
Masculino      64934
Femenino       43324
Desconocido     4668
empresa         1333
Name: count, dtype: int64


In [15]:
# ... (código anterior para crear la columna 'sexo') ...

# Obtener la cantidad en cada categoría de la columna 'sexo' antes de la redistribución
conteo_sexo = merged_df['sexo'].value_counts()
print("Cantidad en cada categoría de la columna 'sexo' antes de la redistribución:")
print(conteo_sexo)

# Importar numpy
import numpy as np

# Paso 1: Obtener la distribución actual de 'Masculino' y 'Femenino'
df_sexo_conocido = merged_df[merged_df['sexo'].isin(['Masculino', 'Femenino'])]
total_sexo_conocido = len(df_sexo_conocido)
conteo_sexo_conocido = df_sexo_conocido['sexo'].value_counts()

proporcion_masculino = conteo_sexo_conocido['Masculino'] / total_sexo_conocido
proporcion_femenino = conteo_sexo_conocido['Femenino'] / total_sexo_conocido

print("Proporción de 'Masculino':", proporcion_masculino)
print("Proporción de 'Femenino':", proporcion_femenino)

# Verificar que las proporciones sumen 1
suma_proporciones = proporcion_masculino + proporcion_femenino
if not np.isclose(suma_proporciones, 1.0):
    # Ajustar las proporciones para que sumen 1
    proporcion_masculino /= suma_proporciones
    proporcion_femenino /= suma_proporciones

# Paso 2: Redistribuir los 'Desconocido'
mascara_desconocido = merged_df['sexo'] == 'Desconocido'
num_desconocido = mascara_desconocido.sum()

# Generar una asignación aleatoria según las proporciones
asignacion_aleatoria = np.random.choice(
    ['Masculino', 'Femenino'],
    size=num_desconocido,
    p=[proporcion_masculino, proporcion_femenino]
)

# Asignar los nuevos valores a la columna 'sexo'
merged_df.loc[mascara_desconocido, 'sexo'] = asignacion_aleatoria

# Obtener el nuevo conteo de 'sexo'
nuevo_conteo_sexo = merged_df['sexo'].value_counts()
print("Nuevo conteo en cada categoría de la columna 'sexo' después de la redistribución:")
print(nuevo_conteo_sexo)

# Paso final: Guardar el DataFrame resultante en un nuevo archivo Excel
merged_df.to_excel('resultado_final.xlsx', index=False)

print("El nuevo archivo Excel 'resultado_final.xlsx' ha sido generado exitosamente.")


Cantidad en cada categoría de la columna 'sexo' antes de la redistribución:
sexo
Masculino      64934
Femenino       43324
Desconocido     4668
empresa         1333
Name: count, dtype: int64
Proporción de 'Masculino': 0.5998078663932458
Proporción de 'Femenino': 0.4001921336067542
Nuevo conteo en cada categoría de la columna 'sexo' después de la redistribución:
sexo
Masculino    67711
Femenino     45215
empresa       1333
Name: count, dtype: int64
El nuevo archivo Excel 'resultado_final.xlsx' ha sido generado exitosamente.


In [16]:
# Paso final: Guardar el DataFrame resultante en un nuevo archivo Excel llamado 'Coder2.xlsx'
merged_df.to_excel('Coder2.xlsx', index=False)

print("El nuevo archivo Excel 'Coder2.xlsx' ha sido generado exitosamente.")


El nuevo archivo Excel 'Coder2.xlsx' ha sido generado exitosamente.
